In [30]:
from dataclasses import dataclass
from typing import Sequence

@dataclass
class Term: pass

@dataclass
class Var(Term):
    name: str

Environment: TypeAlias = Sequence[tuple[Var,Term]]

@dataclass
class Fun(Term):
    x: Var
    t: Term

@dataclass
class App(Term):
    l: Term
    r: Term

@dataclass
class Value(Term): pass

@dataclass
class Num(Value):
    n: int

@dataclass
class Op(Term):
    op: str
    left: Term
    right: Term

@dataclass
class Ifz(Term):
    cond: Term
    thenTerm: Term
    elseTerm: Term

@dataclass
class Fix(Term):
    x: Var
    t: Term

@dataclass
class Let(Term):
    x: Var
    t: Term
    body: Term

@dataclass
class Closure(Value):
    x: Var
    t: Term
    e: Environment

@dataclass
class Thunk(Term):
    term: Fix
    e: Environment

def searchEnv(x: Var, e: Environment) -> Value:
    for (var, val) in e:
        if var == x:
            match val:
                case Thunk(term, e2):
                    return interp(term, e2)
                case _: return val
    raise Exception("Unbound variable: " + str(x))

def extendEnv(x: Var, t: Term, e: Environment) -> Environment:
    return [(x, t)] + e

def checkNumber(num: Value) -> int:
    match num:
        case Num(n): return n
    raise Exception("Not a number: " + str(num))

def interp(t: Term, e: Environment) -> Value:
    match t:
        case Var(x): return searchEnv(t, e)
        case App(t, u):
            w = interp(u, e)
            match interp(t, e):
                case Closure(x, t2, e2):
                    return interp(t2, extendEnv(x, w, e2))
                case _: raise Exception("Illegal function: " + str(v))
        case Fun(x, t2): return Closure(x, t2, e)
        case Num(n): return t
        case Op(op, l, r):
            lv = checkNumber(interp(l, e))
            rv = checkNumber(interp(r, e))
            match op:
                case '+': return Num(lv + rv)
                case '-': return Num(lv - rv)
                case '*': return Num(lv * rv)
                case '/': return Num(lv / rv)
                case '%': return Num(lv % rv)
                case '<': return Num(1 if lv < rv else 0)
                case '=': return Num(1 if lv == rv else 0)
                case _: raise Exception("Unknown op: " + str(op))
        case Ifz(cond, t, u):
            c = interp(cond, e)
            match c:
                case Num(0): return interp(t, e)
                case Num(_): return interp(u, e)
                case _: raise Exception("Condition not a number: " + str(c))
        case Fix(x, t2):
            return interp(t2, extendEnv(x, Thunk(t, e), e))
        case Let(x, t, u):
            w = interp(t, e)
            return interp(u, extendEnv(x, w, e))

print(interp(Op('+', Num(1), Num(2)), []))  # Numeric expression
print(interp(Var('x'), [(Var('x'), Num(1))]))  # Variable
print(interp(App(Fun(Var('x'), Op('*', Var('x'), Var('x'))), Num(3)), [])) # Function and application
print(interp(App(Fix(Var('f'), Fun(Var('x'), Ifz(Var('x'), Num(1), Op('*', Var('x'), App(Var('f'), Op('-', Var('x'), Num(1))))))), Num(10)), [])) # Recursive function call using Fix


Num(n=3)
Num(n=1)
Num(n=9)
Num(n=3628800)


In [32]:
@dataclass
class FixFun(Term):
    f: Var
    x: Var
    t: Term

@dataclass
class RecClosure(Value):
    f: Var
    x: Var
    t: Term
    e: Environment

def searchEnv(x: Var, e: Environment) -> Value:
    for (var, val) in e:
        if var == x:
            return val
    raise Exception("Unbound variable: " + str(x))

def interp(t: Term, e: Environment) -> Value:
    match t:
        case Var(x): return searchEnv(t, e)
        case App(t, u):
            w = interp(u, e)
            v = interp(t, e)
            match v:
                case RecClosure(f, x, t2, e2):
                    return interp(t2, extendEnv(x, w, extendEnv(f, v, e2)))
                case _: raise Exception("Illegal function: " + str(v))
        case Fun(x, t2): return RecClosure(Var(''), x, t2, e)
        case Num(n): return t
        case Op(op, l, r):
            lv = checkNumber(interp(l, e))
            rv = checkNumber(interp(r, e))
            match op:
                case '+': return Num(lv + rv)
                case '-': return Num(lv - rv)
                case '*': return Num(lv * rv)
                case '/': return Num(lv / rv)
                case '%': return Num(lv % rv)
                case '<': return Num(1 if lv < rv else 0)
                case '=': return Num(1 if lv == rv else 0)
                case _: raise Exception("Unknown op: " + str(op))
        case Ifz(cond, t, u):
            c = interp(cond, e)
            match c:
                case Num(0): return interp(t, e)
                case Num(_): return interp(u, e)
                case _: raise Exception("Condition not a number: " + str(c))
        case FixFun(f, x, t2):
            return RecClosure(f, x, t2, e)
        case Let(x, t, u):
            w = interp(t, e)
            return interp(u, extendEnv(x, w, e))

print(interp(App(FixFun(Var('f'), Var('x'), Ifz(Var('x'), Num(1), Op('*', Var('x'), App(Var('f'), Op('-', Var('x'), Num(1)))))), Num(10)), [])) # Recursive function call using FixFun


Num(n=3628800)


In [52]:
from dataclasses import dataclass
from typing import Sequence
from typing import Union

@dataclass
class Instruction: pass

@dataclass
class Ldi(Instruction): 
    n: int

@dataclass
class Push(Instruction): pass

@dataclass
class Extend(Instruction): pass

@dataclass
class Search(Instruction):
    n: int

@dataclass
class Pushenv(Instruction): pass

@dataclass
class Popenv(Instruction): pass

@dataclass
class Mkclos(Instruction):
    i: Sequence[code]

@dataclass
class Apply(Instruction): pass

@dataclass
class Test(Instruction):
    i: Sequence[code]
    j: Sequence[code]

@dataclass
class Add(Instruction): pass

@dataclass
class Sub(Instruction): pass

@dataclass
class Mult(Instruction): pass

@dataclass
class Div(Instruction): pass

@dataclass
class MachineClosure(Value):
    i: Sequence[Code]
    e: Sequence[value]

Stack: TypeAlias = Sequence[Value | Sequence[Value]]

def machine(acc: Value, stack: Stack, env: Sequence[Value], code: Sequence[Instruction]) -> Value:
    while code:
        insn = code.pop(0)
        match insn:
            case Mkclos(i):
                acc = MachineClosure(i, env)
            case Push():
                stack.append(acc)
            case Extend():
                env.append(acc)
            case Search(n):
                acc = env[-n-1]
            case Pushenv():
                stack.append(env)
            case Popenv():
                env = stack.pop()
            case Apply():
                w = stack.pop()
                match acc:
                    case MachineClosure(i, env):
                        env.append(acc)
                        env.append(w)
                        code = i + code
                    case _:
                        raise Exception("Not a closure: " + str(acc))
            case Ldi(n):
                acc = Num(n)
            case Add():
                m = stack.pop(); acc = Num(acc.n + m.n)
            case Sub():
                m = stack.pop(); acc = Num(acc.n - m.n)
            case Mult():
                m = stack.pop(); acc = Num(acc.n * m.n)
            case Div():
                m = stack.pop(); acc = Num(acc.n / m.n)
            case Test(i, j):
                match acc:
                    case Num(0):
                        code = i + code
                    case Num(_):
                        code = j + code
                    case _:
                        raise Exception("Not a number: " + str(acc))
    return acc

print(machine(Num(100), [], [], []))  # -> 100
print(machine(Num(100), [Num(200)], [], [Add()])) # -> 300
print(machine(Num(333), [], [], [Push(), Add()])) # -> 666
print(machine(Num(3), [], [], [Extend(), Search(0)])) # -> 3
print(machine(Num(0), [], [], [Test([Ldi(1)], [Ldi(2)])])) # -> 1
print(machine(Num(1), [], [], [Test([Ldi(1)], [Ldi(2)])])) # -> 2

fact6 = [Pushenv(),
 Mkclos([Search(0),
	Test([Ldi(1)],
	     [Pushenv(),
	      Ldi(1),
	      Push(),
	      Search(0),
	      Sub(),
	      Push(),
	      Search(1),
	      Apply(),
	      Popenv(),
	      Push(),
	      Search(0),
	      Mult()])]),
 Extend(),
 Pushenv(),
 Ldi(6),
 Push(),
 Search(0),
 Apply(),
 Popenv(),
 Popenv()]

print(str(fact6))

def compile(t: term, env: Sequence[Var]) -> Sequence[Code]:
    match t:
        case Var(name):
            for (index, v) in enumerate(env):
                if v.name == name:
                    return [Search(index)]
            raise Exception("Unbound variable: " + name)
        case App(t, u):
            return [Pushenv()] + compile(u, env) + [Push()] + compile(t, env) + [Apply(), Popenv()]
        case Fun(x, t):
            return [Mkclos(compile(t, [x, Var('')] + env))]
        case FixFun(f, x, t):
            return [Mkclos(compile(t, [x, f] + env))]
        case Num(n):
            return [Ldi(n)]
        case Op(op, t, u):
            dic = {'+': Add(), '-': Sub(), '*': Mult(), '/': Div()}
            return compile(u, env) + [Push()] + compile(t, env) + [dic[op]]
        case Ifz(t, u, v):
            return compile(t, env) + [Test(compile(u, env), compile(v, env))]
        case Let(x, t, u):
            return [Pushenv()] + compile(t, env) + [Extend()] + compile(u, [x] + env) + [Popenv()]
        case _:
            raise Exception("Illegal term: " + str(t))
    
compile(Let(Var('f'), 
                FixFun(Var('f'), Var('x'), 
                       Ifz(Var('x'), Num(1), 
                            Op('*', Var('x'),
                                    App(Var('f'), Op('-', Var('x'), Num(1)))))),
                App(Var('f'), Num(6))), [])

print(machine(Num(-1), [], [], compile(App(Fun(Var('x'), Var('x')),Num(1)),[]))) # => 1
print(machine(Num(-1), [], [], compile(App(Fun(Var('x'), Op('*', Var('x'), Var('x'))),Num(3)),[]))) # => 9

print(machine(Num(-1), [], [], compile(App(App(FixFun(Var('f'),Var('x'), Fun(Var('y'),Op('*', Var('x'), Var('y')))),Num(2)),Num(3)),[]))) # => 6

factcall = compile(Let(Var('f'), 
                FixFun(Var('f'), Var('x'), 
                       Ifz(Var('x'), Num(1), 
                            Op('*', Var('x'),
                                    App(Var('f'), Op('-', Var('x'), Num(1)))))),
                App(Var('f'), Num(1))), [])
print(machine(Num(0), [], [], factcall))

Num(n=100)
Num(n=300)
Num(n=666)
Num(n=3)
Num(n=1)
Num(n=2)
[Pushenv(), Mkclos(i=[Search(n=0), Test(i=[Ldi(n=1)], j=[Pushenv(), Ldi(n=1), Push(), Search(n=0), Sub(), Push(), Search(n=1), Apply(), Popenv(), Push(), Search(n=0), Mult()])]), Extend(), Pushenv(), Ldi(n=6), Push(), Search(n=0), Apply(), Popenv(), Popenv()]
Num(n=1)
Num(n=9)
Num(n=6)
Num(n=0)
